# Profiling loading of the datasets (which comprehends computation of analysis variables)

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from parameters_dict import physics_parameters
from dark_nus_utils import load_datasets
from plot_utils import set_plot_title, annotated_2d_plot
from exp_analysis_class import exp_analysis, gamma_general

## Declaring object

In [3]:
this_exp_analyis = exp_analysis(hierarchy='light', D_or_M='majorana')
m4, mz = 0.8, 0.03

## Profiling df_base

In [19]:
%lprun -f this_exp_analyis.load_df_base this_exp_analyis.load_df_base(1000000)

loading df base
initialising df base


Timer unit: 1e-06 s

Total time: 24.8933 s
File: /n/home00/nfoppiani/projects/dark_nus/nicgen/exp_analysis/exp_analysis_class.py
Function: load_df_base at line 102

Line #      Hits         Time  Per Hit   % Time  Line Contents
   102                                               def load_df_base(self, n_evt=1000000, filename=None):
   103         1          5.0      5.0      0.0          self.n_evt = n_evt
   104         1        475.0    475.0      0.0          print("loading df base")
   105         1          2.0      2.0      0.0          if filename is None:
   106         1     244752.0 244752.0      1.0              self.df_base = pd.read_pickle(f'{self.base_folder}scan/{self.hierarchy}_{self.D_or_M}/{self.m4_limits[0]}_m4_{self.m4_limits[1]}_{self.mz_limits[0]}_mzprime_{self.mz_limits[1]}_nevt_{self.n_evt}.pckl')
   107                                                   else:
   108                                                       self.df_base = pd.read_pickle(filename)
  

In [20]:
%lprun -f this_exp_analyis.initialise_df this_exp_analyis.load_df_base(1000000)

loading df base
initialising df base


Timer unit: 1e-06 s

Total time: 23.6237 s
File: /n/home00/nfoppiani/projects/dark_nus/nicgen/exp_analysis/exp_analysis_class.py
Function: initialise_df at line 125

Line #      Hits         Time  Per Hit   % Time  Line Contents
   125                                               def initialise_df(self, df, which_scan=None):
   126         1   16095581.0 16095581.0     68.1          self.compute_analysis_variables(df)
   127         1     646202.0 646202.0      2.7          self.compute_actual_weights(df, which_scan)
   128         1     708440.0 708440.0      3.0          self.compute_interaction_point(df)
   129         1     269699.0 269699.0      1.1          self.unitary_decay_length(df)
   130         1      31002.0  31002.0      0.1          self.compute_selection(df)
   131                                                   
   132                                                   # flatten index of pandas multiindex
   133         1        474.0    474.0      0.0          df.c

In [21]:
%lprun -f this_exp_analyis.compute_analysis_variables this_exp_analyis.load_df_base(1000000)

loading df base
initialising df base


Timer unit: 1e-06 s

Total time: 16.0883 s
File: /n/home00/nfoppiani/projects/dark_nus/nicgen/exp_analysis/exp_analysis_class.py
Function: compute_analysis_variables at line 146

Line #      Hits         Time  Per Hit   % Time  Line Contents
   146                                               @staticmethod
   147                                               def compute_analysis_variables(df):
   148         5         13.0      2.6      0.0          for comp in ['t','x','y','z']:
   149         4     179609.0  44902.2      1.1              df['pee', comp] = df['plm', comp] + df['plp', comp]
   150         4     327800.0  81950.0      2.0              df['pdark', comp] = df['plm', comp] + df['plp', comp] + df['pnu', comp]
   151                                           
   152         1    2047538.0 2047538.0     12.7          df['recoil_mass', ''] = inv_mass(df['pHad']).round(6)
   153                                                   # e+e- cone vars
   154         1     761626.0 76

## Profiling another df

In [22]:
%lprun -f this_exp_analyis.load_df this_exp_analyis.load_df(0.4, 0.01)

loading df 0.4, 0.01
initialising df 0.4, 0.01


Timer unit: 1e-06 s

Total time: 25.8782 s
File: /n/home00/nfoppiani/projects/dark_nus/nicgen/exp_analysis/exp_analysis_class.py
Function: load_df at line 112

Line #      Hits         Time  Per Hit   % Time  Line Contents
   112                                               def load_df(self, m4, mz):
   113         1        304.0    304.0      0.0          print(f"loading df {m4}, {mz}")
   114         1     253824.0 253824.0      1.0          self.dfs[(m4, mz)] = pd.read_pickle(f'{self.base_folder}m4_{m4}_mzprime_{mz}_{self.hierarchy}_{self.D_or_M}/MC_m4_{m4}_mzprime_{mz}.pckl')
   115         1         87.0     87.0      0.0          print(f"initialising df {m4}, {mz}")
   116         1   25623963.0 25623963.0     99.0          self.initialise_df(self.dfs[(m4, mz)])

In [23]:
%lprun -f this_exp_analyis.initialise_df this_exp_analyis.load_df(0.4, 0.01)

loading df 0.4, 0.01
initialising df 0.4, 0.01


Timer unit: 1e-06 s

Total time: 25.5641 s
File: /n/home00/nfoppiani/projects/dark_nus/nicgen/exp_analysis/exp_analysis_class.py
Function: initialise_df at line 125

Line #      Hits         Time  Per Hit   % Time  Line Contents
   125                                               def initialise_df(self, df, which_scan=None):
   126         1   17729000.0 17729000.0     69.4          self.compute_analysis_variables(df)
   127         1     383401.0 383401.0      1.5          self.compute_actual_weights(df, which_scan)
   128         1     844931.0 844931.0      3.3          self.compute_interaction_point(df)
   129         1     303545.0 303545.0      1.2          self.unitary_decay_length(df)
   130         1      32319.0  32319.0      0.1          self.compute_selection(df)
   131                                                   
   132                                                   # flatten index of pandas multiindex
   133         1        502.0    502.0      0.0          df.c

In [24]:
%lprun -f this_exp_analyis.compute_analysis_variables this_exp_analyis.load_df(0.4, 0.01)

loading df 0.4, 0.01
initialising df 0.4, 0.01


Timer unit: 1e-06 s

Total time: 17.7265 s
File: /n/home00/nfoppiani/projects/dark_nus/nicgen/exp_analysis/exp_analysis_class.py
Function: compute_analysis_variables at line 146

Line #      Hits         Time  Per Hit   % Time  Line Contents
   146                                               @staticmethod
   147                                               def compute_analysis_variables(df):
   148         5         13.0      2.6      0.0          for comp in ['t','x','y','z']:
   149         4     190352.0  47588.0      1.1              df['pee', comp] = df['plm', comp] + df['plp', comp]
   150         4     345103.0  86275.8      1.9              df['pdark', comp] = df['plm', comp] + df['plp', comp] + df['pnu', comp]
   151                                           
   152         1    2196953.0 2196953.0     12.4          df['recoil_mass', ''] = inv_mass(df['pHad']).round(6)
   153                                                   # e+e- cone vars
   154         1     831692.0 83